In [31]:
import pandas as pd
import numpy as np
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from typing import Tuple, List, NamedTuple

In [3]:
CLIENT_ID = os.environ["SPOTIFY_CLIENT_ID"]
CLIENT_SECRET = os.environ["SPOTIFY_CLIENT_SECRET"]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri='http://example.com', scope="playlist-modify-public"))

track_name = "Save Your Tears"
artist_name = "Weeknd"

a = sp.search(q=f"track: {track_name} artist: {artist_name}", type='track', limit=1)

Enter the URL you were redirected to:  https://example.com/?code=AQAAO8oqzOHBgcWRzrQroTFqQ3Hb8VvLcTb73rJPe4xOZQLvuNgSdb7oqr_22tNQLCwjhVffEkV_l5KAt7hS_r0d5BAbwYHRmvWgVSgvcylDPy_F2nZS_Qu5H93JiTvsczdLJB27AMccwtVx534WeD8hwM5eTQlQCpyzeRAkmJgIY4oy72RL9G85ZfGBZg


In [4]:
a["tracks"]["items"][0]["id"]

'5QO79kh1waicV47BqGRL3g'

In [5]:
a["tracks"]["items"][0]["album"]["release_date"]

'2020-03-20'

In [23]:
a

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A+Save+Your+Tears+artist%3A+Weeknd&type=track&offset=0&limit=1',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'},
       'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ',
       'id': '1Xyo4u8uXC1ZmMpatF05PJ',
       'name': 'The Weeknd',
       'type': 'artist',
       'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}],
     'available_markets': ['AR',
      'AU',
      'AT',
      'BE',
      'BO',
      'BR',
      'BG',
      'CA',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DK',
      'DO',
      'DE',
      'EC',
      'EE',
      'SV',
      'FI',
      'FR',
      'GR',
      'GT',
      'HN',
      'HK',
      'HU',
      'IS',
      'IE',
      'IT',
      'LV',
      'LT',
      'LU',
      'MY',
      'MT',
      'MX',
      'NL',
      'NZ',
      'NI',
      'NO',


In [6]:
ENDPOINT_CHARTS = "https://www.officialcharts.com/charts/singles-chart/%s/7501/"

def generate_dates(week_gap: int = 2, years_back: int = 50):

    today = datetime.now()
    start_date = today - timedelta(days=365*years_back)
    
    for n in range(int((today - start_date).days)):
        yield start_date + timedelta(days=n*week_gap*7)
    
for single_date in generate_dates():
    print(single_date.strftime("%Y%m%d"))

19740713
19740727
19740810
19740824
19740907
19740921
19741005
19741019
19741102
19741116
19741130
19741214
19741228
19750111
19750125
19750208
19750222
19750308
19750322
19750405
19750419
19750503
19750517
19750531
19750614
19750628
19750712
19750726
19750809
19750823
19750906
19750920
19751004
19751018
19751101
19751115
19751129
19751213
19751227
19760110
19760124
19760207
19760221
19760306
19760320
19760403
19760417
19760501
19760515
19760529
19760612
19760626
19760710
19760724
19760807
19760821
19760904
19760918
19761002
19761016
19761030
19761113
19761127
19761211
19761225
19770108
19770122
19770205
19770219
19770305
19770319
19770402
19770416
19770430
19770514
19770528
19770611
19770625
19770709
19770723
19770806
19770820
19770903
19770917
19771001
19771015
19771029
19771112
19771126
19771210
19771224
19780107
19780121
19780204
19780218
19780304
19780318
19780401
19780415
19780429
19780513
19780527
19780610
19780624
19780708
19780722
19780805
19780819
19780902
19780916
19780930
1

In [7]:
response = requests.get(ENDPOINT_CHARTS % "19800101")
html = response.text

In [10]:
soup = BeautifulSoup(html, "html.parser")

In [11]:
items = soup.find_all("div", {"class": "chart-item"})

In [19]:
song = items[0].find("a", {"class": "chart-name"})
artist = items[0].find("a", {"class": "chart-artist"})

In [23]:
song.find("span", {"class": None})

<span>ANOTHER BRICK IN THE WALL</span>

In [24]:
artist.find("span", {"class": None})`

<span>PINK FLOYD</span>

In [35]:
class SongInfo:

    def __init__(self, artist: str, song: str):
        self.artist = artist
        self.song = song

    def __eq__(self, other) -> bool:
        if isinstance(other, SongInfo):
            return all([self.artist == other.artist, self.song == other.song])
        return False


class SongsContainer:

    def __init__(self):
        self.songs = []


    def add_song(self, song: SongInfo) -> None:
        if song not in self.songs and isinstance(song, SongInfo):
            self.songs.append(song)


        

In [27]:
def retrieve_top_songs(date: str) -> List[Tuple[str, str]]:
    try:
        response = requests.get(ENDPOINT_CHARTS % date)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.find_all("div", {"class": "chart-item"})

        for item in items:
            song = item.find("a", {"class": "chart-name"}).find("span", {"class": None})
            artist = item.find("a", {"class": "chart-artist"}).find("span", {"class": None})

            return song.text, artist.text
    except Exception as exception:
        print(exception)

In [30]:
retrieve_top_songs("20040309")

('TOXIC', 'BRITNEY SPEARS')